Data Processing

In [ ]:

import pandas as pd
df = pd.read_csv('/work/final_forreal.csv')

home_ohe = pd.get_dummies(df['homeType'])
home_ohe
df = pd.concat([df, home_ohe], axis = 1)
df

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
#columns=['Lead', 'Address', 'city', 'cityId', 'county', 'countyFIPS', 'homeType', 'livingArea', 'livingAreaValue', 'parcelId', 'price', 'regionString', 'state', 'stateId', 'zipcode']

X = df.drop(columns=['Lead', 'Address', 'city', 'cityId', 'county', 'countyFIPS', 'homeType', 'livingArea', 'livingAreaValue', 'parcelId', 'price', 'regionString', 'state', 'stateId', 'zipcode'])
y = df['Lead']

# Replace NaN values with the mean of the non-missing values
imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Use SMOTE to oversample the positive class to 30,000 samples
smote = SMOTE(sampling_strategy={1: 50000}, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

Model

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

# LightGBM model with combined data
lgb_model = lgb.LGBMClassifier(
    random_state=42,
    metric='binary_error'  # Note: LightGBM uses binary_error for binary classification
)

# Train the model on the resampled training data
lgb_model.fit(X_train_resampled, y_train_resampled, eval_set=[(X_test, y_test)])

# Get class probabilities for the test data
probabilities = lgb_model.predict_proba(X_test)

# Calculate AUROC on the test data
auroc = roc_auc_score(y_test, probabilities[:, 1])

# Make predictions on the test data
lgb_predict = lgb_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, lgb_predict)

# Calculate false-negative rate
conf_matrix = confusion_matrix(y_test, lgb_predict)
false_negative_rate = conf_matrix[1, 0] / (conf_matrix[1, 0] + conf_matrix[1, 1])

# Print metrics
print('AUROC:', auroc)
print('Accuracy:', accuracy)
print('False Negative Rate:', false_negative_rate)